In [0]:
%sh git clone https://github.com/ammaryasirnaich/apache_spark_stream_lab.git

In [0]:
%sh ls apache_spark_stream_lab

In [0]:
import os
import socket
import os
from _thread import *
import time


In [0]:
def main():
    ServerSideSocket = socket.socket()
    dataset_path= os.getcwd()+"/apache_spark_stream_lab/nasaAccessLogs.csv"
    file_exists = os.path.exists(dataset_path)
    if (file_exists==False):
      print("Dataset path Error")
      return
    else:    
      print("Dataset path accessible:"+str(file_exists))
      
    host = '127.0.0.1'
    port = 5551
    ThreadCount = 0
    try:
        ServerSideSocket.bind((host, port))
    except socket.error as e:
        print(str(e))
    print('Socket is listening..')
    ServerSideSocket.listen(5)

    def multi_threaded_client(connection):
        connection.send(str.encode('Server is working:')) 
        file_handle = open(dataset_path, 'r')
       
        with open(dataset_path, 'r') as file_handle:
            while True:
                try:
                    line = file_handle.readline()
                except: 
                    connection.close()
                    file_handle.close()   
                
                if line != "":  
                    try:
                        print(line, end='')
                        connection.sendall(line.encode())      
                    except socket.error:
                        connection.close()
                        file_handle.close()
                else:
                    file_handle.seek(0)
                time.sleep(1)

            connection.close()
    while True:
        Client, address = ServerSideSocket.accept()
        print('Connected to: ' + address[0] + ':' + str(address[1]))
        start_new_thread(multi_threaded_client, (Client, ))
        ThreadCount += 1
        print('Thread Number: ' + str(ThreadCount))
    ServerSideSocket.close()


In [0]:
#start the server
main()

java.net.NoRouteToHostException: No route to host
	at java.base/sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at java.base/sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:779)
	at shaded.v9_4.org.eclipse.jetty.io.SelectorManager.doFinishConnect(SelectorManager.java:355)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:232)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1400(ManagedSelector.java:62)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:543)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:401)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:360)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:184)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:171)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:129)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:367)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.run(InstrumentedQueuedThreadPool.scala:131)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:782)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:914)
	at java.base/java.lang.Thread.run(Thread.java:834)

In [0]:
%sh fuser -k 5551/tcp